In [ ]:
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
class NeuralMachineTranslation:
    def __init__(self, max_sequence_length=10):
        """
        Initialize Neural Machine Translation Model

        Args:
            max_sequence_length (int): Maximum length of input/output sequences
        """
        self.max_sequence_length = max_sequence_length
        self.input_tokenizer = None
        self.output_tokenizer = None
        self.model = None

    def prepare_data(self, input_texts, output_texts):
        """
        Prepare input and output texts for training

        Args:
            input_texts (list): List of input language sentences
            output_texts (list): List of output language sentences

        Returns:
            tuple: Tokenized and padded input and output sequences
        """
        # Tokenize input texts
        self.input_tokenizer = Tokenizer(filters='', lower=False)
        self.input_tokenizer.fit_on_texts(input_texts)
        input_sequences = self.input_tokenizer.texts_to_sequences(input_texts)
        encoder_input_data = pad_sequences(input_sequences, maxlen=self.max_sequence_length, padding='post')

        # Tokenize output texts
        self.output_tokenizer = Tokenizer(filters='', lower=False)
        self.output_tokenizer.fit_on_texts(output_texts)
        output_sequences = self.output_tokenizer.texts_to_sequences(output_texts)
        decoder_input_data = pad_sequences(output_sequences, maxlen=self.max_sequence_length, padding='post')

        # One-hot encode output sequences
        # Shift the target data by one time step
        decoder_target_data = np.zeros((len(output_texts), self.max_sequence_length,
                                        len(self.output_tokenizer.word_index) + 1), dtype='float32')

        for i, sequence in enumerate(decoder_input_data):
            for t, word_index in enumerate(sequence):
                if word_index > 0 and t > 0:
                    decoder_target_data[i, t-1, word_index] = 1.0

        # Get vocabulary sizes
        input_vocab_size = len(self.input_tokenizer.word_index) + 1
        output_vocab_size = len(self.output_tokenizer.word_index) + 1

        return (encoder_input_data, decoder_input_data, decoder_target_data,
                input_vocab_size, output_vocab_size)

    def build_model(self, input_vocab_size, output_vocab_size):
        """
        Build the Neural Machine Translation model

        Args:
            input_vocab_size (int): Size of input language vocabulary
            output_vocab_size (int): Size of output language vocabulary
        """
        # Encoder
        encoder_inputs = tf.keras.layers.Input(shape=(self.max_sequence_length,))
        encoder_embedding = tf.keras.layers.Embedding(
            input_vocab_size, 256, mask_zero=True)(encoder_inputs)
        encoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
        encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
        encoder_states = [state_h, state_c]

        # Decoder
        decoder_inputs = tf.keras.layers.Input(shape=(self.max_sequence_length,))
        decoder_embedding = tf.keras.layers.Embedding(
            output_vocab_size, 256, mask_zero=True)(decoder_inputs)
        decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(
            decoder_embedding, initial_state=encoder_states)

        # Dense output layer
        decoder_dense = tf.keras.layers.Dense(
            output_vocab_size, activation='softmax')
        decoder_outputs = decoder_dense(decoder_outputs)

        # Compile the model
        model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
        model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

        self.model = model

    def train(self, input_texts, output_texts, epochs=50, batch_size=32):
        """
        Train the Neural Machine Translation model

        Args:
            input_texts (list): Training input language sentences
            output_texts (list): Training output language sentences
            epochs (int): Number of training epochs
            batch_size (int): Batch size for training
        """
        # Prepare data
        (encoder_input_data, decoder_input_data,
         decoder_target_data, input_vocab_size,
         output_vocab_size) = self.prepare_data(input_texts, output_texts)

        # Build model
        self.build_model(input_vocab_size, output_vocab_size)

        # Train the model
        self.model.fit(
            [encoder_input_data, decoder_input_data],
            decoder_target_data,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2
        )

    def save(self, filepath):
        """
        Save the entire model to a specified filepath
        """
        self.model.save(filepath, save_format='h5')

    def translate(self, input_sentence):
        """
        Translate a single sentence

        Args:
            input_sentence (str): Sentence to translate

        Returns:
            str: Translated sentence
        """
        if not self.model or not self.input_tokenizer or not self.output_tokenizer:
            raise ValueError("Model must be trained before translation")

        # Tokenize and pad input sentence
        input_seq = self.input_tokenizer.texts_to_sequences([input_sentence])
        input_padded = pad_sequences(input_seq, maxlen=self.max_sequence_length, padding='post')

        # Prepare decoder input
        decoder_input = np.zeros((1, self.max_sequence_length))
        decoder_input[0, 0] = self.output_tokenizer.word_index.get('<start>', 1)  # Start token

        # Translation process
        translated_words = []
        for i in range(self.max_sequence_length):
            # Predict next word
            output = self.model.predict([input_padded, decoder_input])

            # Get the index of the word with the highest probability
            predicted_word_index = np.argmax(output[0, i, :])

            # Convert index to word
            predicted_word = self.output_tokenizer.index_word.get(predicted_word_index, '')

            # Stop if no word is predicted or we've reached max length
            if not predicted_word or predicted_word == '<end>':
                break

            translated_words.append(predicted_word)

            # Update decoder input
            decoder_input[0, i+1] = predicted_word_index

        return ' '.join(translated_words)

In [3]:
def read_texts_from_file(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            # Strip whitespace and remove empty lines
            texts = ['<start> ' + line.strip() + ' <end>' for line in file if line.strip()]
        return texts
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
        return []
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return []

In [ ]:
def main():

    model_save_path = 'sp_en_model.h5'


    input_texts = read_texts_from_file('datasets/spanish')
    output_texts = read_texts_from_file('datasets/eng_sp.en')

    input_texts = input_texts[:10000]
    output_texts = output_texts[:10000]

    # Initialize and train the model
    nmt_model = NeuralMachineTranslation(max_sequence_length=5)

    # Train the model
    nmt_model.train(input_texts, output_texts, epochs=100)
    nmt_model.save(model_save_path)

    with open("sp_en_input_tokenizer.pkl", "wb") as f:
        pickle.dump(nmt_model.input_tokenizer, f)

    with open("sp_en_output_tokenizer.pkl", "wb") as f:
        pickle.dump(nmt_model.output_tokenizer, f)



    # Translate the first sentence
    if input_texts:
        input_sentence = input_texts[0].replace('<start>', '').replace('<end>', '').strip()
        translation = nmt_model.translate(input_sentence)
        print(f"Input: {input_sentence}")
        print(f"Translation: {translation}")
    else:
        print("No input texts available for translation.")

if __name__ == "__main__":
    main()

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - accuracy: 0.2019 - loss: 5.5134 - val_accuracy: 0.2245 - val_loss: 4.4255
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.2306 - loss: 4.2235 - val_accuracy: 0.2309 - val_loss: 4.3122
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.2343 - loss: 4.0986 - val_accuracy: 0.2344 - val_loss: 4.2411
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.2365 - loss: 4.0185 - val_accuracy: 0.2378 - val_loss: 4.1776
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.2415 - loss: 3.9255 - val_accuracy: 0.2404 - val_loss: 4.1152
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.2471 - loss: 3.8302 - val_accuracy: 0.2378 - val_loss: 4.0700
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.2523 - loss: 3.7318 - val_accuracy: 0.2401 - val_loss: 4.0393
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 28ms/step - accuracy: 0.2577 - loss: 3.6670